In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")   # подавляем предупреждения

In [ ]:
# Считываем данные в указанные переменные train_data и test_data.
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
# Чтобы посмотреть на данные, достаточно ввести в ячейку название переменной, которая содержит датафрейм.

train_data

In [ ]:
test_data

In [ ]:
all_data = pd.concat([train_data, test_data], ignore_index=True, sort=False)
# all_data.drop('Survived', axis=1, inplace=True)

all_data

In [ ]:
# функция, определяющая соотношение пропущенных данных
def check_missing_data(all_data):
  missing_data = {}
  for key in all_data.columns:
      if all_data[key].isnull().sum() > 0:
          missing_data[key] = (all_data[key].isnull().sum()  /  len(all_data[key])) * 100
  return missing_data

missing_data = check_missing_data(all_data)
missing_data = pd.DataFrame(missing_data, index=['MissingValues']).T.sort_values(by='MissingValues',ascending=False)

missing_data

In [ ]:
# Cabin - плохой атрибут, выкидываем
all_data.drop('Cabin',axis=1, inplace=True)

In [ ]:
all_data

In [ ]:
# зависимости атрибутов друг между другом
# all_data.corr()

In [ ]:
# можно также просмотреть взаимную корреляцию двух датафреймов или серий
# all_data.corrwith(all_data)

In [ ]:
# определяем сущности/сэмплы, для которых необходимо указать возраст (с учетом сильной зависимости возраста от класса)

mask1 = (all_data.Pclass == 1) & (all_data.Age.isnull())
mask2 = (all_data.Pclass == 2) & (all_data.Age.isnull())
mask3 = (all_data.Pclass == 3) & (all_data.Age.isnull())

In [ ]:
# в пустые значения атрибута "Возраст" помещаем значение, равное наиболее распространенному среди всех объектов данного Pclass'а

all_data.loc[mask1, 'Age'] = all_data[all_data.Pclass == 1]['Age'].median()
all_data.loc[mask2, 'Age'] = all_data[all_data.Pclass == 2]['Age'].median()
all_data.loc[mask3, 'Age'] = all_data[all_data.Pclass == 3]['Age'].median()

In [ ]:
all_data['Age'].isna().any()

# np.False_ - круто!!

In [ ]:
missing_data = check_missing_data(all_data)
missing_data = pd.DataFrame(missing_data, index=['MissingValues']).T.sort_values(by='MissingValues',ascending=False)
missing_data

# Cabin выкинули, Age заполнили -> они пропали

In [ ]:
all_data[all_data.Fare.isnull()]

In [ ]:
all_data.Fare.fillna(all_data.Fare[all_data.Pclass == 3].median(), inplace=True)
# nani

# mask4 = all_data.Fare.isnull()

# all_data.loc[mask4, 'Fare'] = all_data[all_data.Pclass == 3]['Fare'].median()

In [ ]:
all_data[all_data.Fare.isnull()]

In [ ]:
all_data['Embarked'].describe()
all_data.Embarked.fillna('S', inplace=True)

In [ ]:
all_data.Name.astype("category")

In [ ]:
# .cat.as_ordered() позволяет преобразовать категориальный признак в порядковый (задать порядок на множестве категорий этого признака)

all_data.Name.astype("category").cat.as_ordered()

In [ ]:
# будем менять строчный тип на тип категории
all_data_copy = all_data.copy()

# конвертируем тип данных объектов в категориальный тип данных
for key,value in all_data_copy.items():
    if pd.api.types.is_string_dtype(value):
        all_data_copy[key] = value.astype("category").cat.as_ordered()

In [ ]:
# заменяем категории на их код
for key, value in all_data_copy.items():
     if not pd.api.types.is_numeric_dtype(value):
       all_data_copy[key] = pd.Categorical(value).codes + 1

# если тип данных столбца не числовой, преобразуем его значения в категории и заменяем их на числовые коды категорий + 1

In [ ]:
all_data_copy

In [ ]:
all_data_copy.corr()

In [ ]:
#получаем число уникальных значений для каждого столбца
for column in all_data:
    print(column + '\t\t', len(all_data[column].unique()))

In [ ]:
# удалим PassengerId, в нем нет смысла

all_data.drop('PassengerId', axis=1, inplace=True)

In [ ]:
# делаем все строки для имен в нижнем регистре
for col in all_data.columns:
    if pd.api.types.is_string_dtype(all_data[col]):
        all_data[col] = all_data[col].str.lower()

In [ ]:
all_data['Name']

In [ ]:
# получаем префиксы имен с помощью применения лямбда-функции (чины людей) и функции apply
def some_func(x):
  return x[x.find(', ') + len(', ') : x.rfind('.')]

all_data['Name_Prefix'] = all_data['Name'].apply(some_func)

all_data['Name_Prefix'] = all_data['Name'].apply(lambda x: x[x.find(', ')+len(', '):x.rfind('.')])

In [ ]:
all_data

In [ ]:
all_data['Name_Prefix'].unique()

In [ ]:
all_data[all_data.Name_Prefix == 'mrs. martin (elizabeth l']

In [ ]:
# корректируем введенные данные с помощью replace

all_data['Name_Prefix'] = all_data['Name_Prefix'].replace("mrs. martin (elizabeth l","mrs")

all_data['Name_Prefix'] = all_data['Name_Prefix'].replace("mlle","miss")    # с французского на английский
all_data['Name_Prefix'] = all_data['Name_Prefix'].replace("mme","mrs")   

all_data['Name_Prefix'] = all_data['Name_Prefix'].replace("don","sir")      # с испанского на английский
all_data['Name_Prefix'] = all_data['Name_Prefix'].replace("dona","mrs") 

In [ ]:
all_data['Name_Prefix'].unique()

In [ ]:
# количество уникальных фамилий
len((all_data["Name"].str.split(",").str.get(0)).unique())

In [ ]:
# разделяем по запятой и берем первую половину, которая по идее содержит фамилию
all_data['Lastname'] = all_data["Name"].str.split(",").str.get(0)

In [ ]:
all_data

In [ ]:
all_data.drop('Name', axis=1, inplace=True)

In [ ]:
all_data.Ticket

In [ ]:
# разделяем значение билета по пробелу и образуем из них два отдельных столбца с данными
all_data['TicketPre'] = all_data.Ticket.apply(lambda x: x.split(' ')[0] if x.isdigit()==False else 'NoPre')
all_data['TicketNum'] = all_data.Ticket.apply(lambda x: x.split(' ')[-1] if x.isdigit()==False else x)

In [ ]:
all_data

In [ ]:
all_data['TicketPre'].unique()

In [ ]:
# получаем значения билетов с одинаковыми префиксами
all_data[['TicketPre','TicketNum']][(all_data.TicketPre=='ston/o2.')|
                                    (all_data.TicketPre=='soton/o2')
                                   ].sort_values(by='TicketNum').head(10)

In [ ]:
# убираем точки и слэши из префиксов
reps = {'.' : '', '/':''}

# .str — доступ к методам строковой обработки pandas
all_data.TicketPre = all_data.TicketPre.str.translate(str.maketrans(reps))

all_data['TicketPre'] = all_data['TicketPre'].replace("sotono2","stono2") 
all_data['TicketPre'] = all_data['TicketPre'].replace("sotonoq","stonoq") 

In [ ]:
all_data.TicketNum.sort_values()

In [ ]:
# смотрим, у кого неправильно введен номер билета 
all_data[all_data['TicketNum']=='line']

In [ ]:
# смотрим, у кого неправильно введен номер билета с учетом фамилии
all_data[['TicketNum','Lastname']][(all_data.Lastname=='johnson')]

In [ ]:
# заменяем line на наиболее часто встречающееся значение
all_data.TicketNum = all_data.TicketNum.replace('line', 347742)

In [ ]:
all_data.TicketNum = all_data.TicketNum.astype(int)

In [ ]:
# смотрим на количество уникальных номеров билетов
len(all_data.TicketNum.unique())

In [ ]:
# будем хранить в словаре первое значение каждой последовательности в качестве ключа и генератор последовательности в качестве значения
# (то есть что-то типа {head : range(FirstNum, LastNum)})

s, head = {}, None
for x in sorted(all_data['TicketNum']):
    # x != s[head].stop - проверяет, является ли текущее значение x следующим после последнего значения текущего диапазона
    # s[head].stop - следующее число за последним в диапазоне
    if head is None or x != s[head].stop:
        head = x
    s[head] = range(head, x + 1) # обновление диапазона

In [ ]:
# делаем функцию, которая для конкретного билета возвращает первое число последовательности (то есть ключ для словаря)
def get_head(ticketNum):
    for head, range in s.items():
        if ticketNum in range:
            x = head   
    return x

In [ ]:
# заменяем в общем дата-фрейме
all_data['TicketNum_Groups'] = all_data['TicketNum'].apply(lambda x: get_head(x))
len(all_data['TicketNum_Groups'].unique())

In [ ]:
# заменяем тип данных, как говорилось ранее
all_data['TicketNum_Groups'] = all_data['TicketNum_Groups'].astype(object)

In [ ]:
all_data

In [ ]:
# удаляем столбцы, которые стали ненужными 
all_data.drop('Ticket', axis=1,inplace=True)
all_data.drop('TicketNum', axis=1,inplace=True)

In [ ]:
all_data

In [ ]:
# смотрим на корелляцию с целевой переменной (только для строк, целевой признак которых нам уже известен)
train_data['PassengerId'].count()   # считаем, сколько у нас таких строк

In [ ]:
# корреляция между целевыми признаком и остальными
# all_data[:891].corrwith(train_data['Survived']).sort_values(ascending=False)

In [ ]:
# новый признак для удобства (в силу схожести смысла признаков SibSp и Parch)
all_data['Relatives'] = all_data['SibSp'] + all_data['Parch']

In [ ]:
all_data

In [ ]:
all_data['Age'].hist()

In [ ]:
all_data['Sex'].hist()

In [ ]:
# all_data.groupby(['Sex', 'Pclass']).median()

In [ ]:
# all_data.groupby('Age').get_group(24)

In [ ]:
all_data

In [ ]:
# Построим распределение людей по возрасту
plt.figure(figsize=(8, 4))
plt.hist(all_data['Age'], label="Age")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.lineplot(x=all_data['Age'], y=all_data['Fare'])     # выдавал ошибку из-за отсутствия x=..., y=...

In [ ]:
# plt.figure(figsize=(12, 6))
# sns.heatmap(all_data.corr(), annot=True, cmap='Greens')

In [ ]:
all_data = all_data.dropna()

In [ ]:
all_data

In [ ]:
# кодируем строковые символы

all_data['Sex'] = pd.factorize(all_data['Sex'])[0]
all_data['Embarked'] = pd.factorize(all_data['Embarked'])[0]
all_data['Name_Prefix'] = pd.factorize(all_data['Name_Prefix'])[0]
all_data['TicketPre'] = pd.factorize(all_data['TicketPre'])[0]
all_data['TicketNum_Groups'] = pd.factorize(all_data['TicketNum_Groups'])[0]
all_data['Lastname'] = pd.factorize(all_data['Lastname'])[0]
# all_data['Survived'] = all_data['Survived'].astype('int')

In [ ]:
# целевой признак в отдельную переменную, убираем из данных, на которых будем обучать

y = all_data['Survived']
all_data.drop(['Survived'], axis=1, inplace=True)

In [ ]:
all_data

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz
from sklearn import tree


# x - признаки, y - целевая переменная
# train_test_split - разделяем данные на тренировочную и тестовую выборки
X_train, X_holdout, y_train, y_holdout = train_test_split(all_data.values, y, test_size=0.3, random_state=17)


clf = DecisionTreeClassifier(max_depth=5, random_state=17)
# knn = KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train, y_train)
# knn.fit(X_train, y_train)


In [ ]:
all_data

In [ ]:
plt.figure(figsize=(50, 20))
# tree.plot_tree(clf, feature_names=X_holdout.feature_names, class_names=y_holdout.target_names, filled=True)
tree.plot_tree(clf, feature_names=all_data.columns, class_names=['Not survived', 'Survived'], filled=True)

plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
tree.plot_tree(clf, feature_names=X_holdout.feature_names, class_names=y_holdout.target_names, filled=True)

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred) # 0.77

In [ ]:
knn_pred = knn.predict(X_holdout)
accuracy_score(y_holdout, knn_pred) # 0.65

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt


# X_train, X_holdout, y_train, y_holdout = train_test_split(all_data.values, y, test_size=0.3, random_state=17)

X, y = all_data.data, all_data.target
clf = DecisionTreeClassifier()
clf.fit(X, y)

plt.figure(figsize=(15, 10))
tree.plot_tree(clf, feature_names=all_data.feature_names, class_names=all_data.target_names, filled=True)
plt.show()